# Agentic RAG Using CrewAI & LangChain

We are going to see how agents can be involved in the RAG system to rtrieve the most relevant information.

In [ ]:
#!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 sentence-transformers langchain-groq --quiet

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools import PDFSearchTool
from crewai.tools  import tool
from crewai import Crew
from crewai import Task
from crewai import Agent

## Load API Keys to environment

In [ ]:
from dotenv import load_dotenv

load_dotenv() # Загружает переменные из .env файла в окружение

print(os.environ['OPENAI_API_BASE'])

## Mention the LLM being used

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4.1-nano",
    temperature=0.1,
    max_tokens=1000,
)

In [ ]:
import requests

pdf_url = 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'
response = requests.get(pdf_url)

with open('attention_is_all_you_need.pdf', 'wb') as file:
    file.write(response.content)

## Create a RAG tool variable to pass our PDF

In [ ]:
rag_tool = PDFSearchTool(pdf='attention_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="openai", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gpt-4.1-nano",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="openai", # or openai, ollama, ...
            config=dict(
                model="text-embedding-3-small",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

In [ ]:
rag_tool.run("How did self-attention mechanism evolve in large language models?")

## Mention the Tavily API Key

In [ ]:
# import os

# Set the Tavily API key
# os.environ['TAVILY_API_KEY'] = 'Add Your Tavily API Key'

In [ ]:
web_search_tool = TavilySearchResults(k=3)

In [ ]:
web_search_tool.run("What is self-attention mechansim in large language models?")

## Define a Tool

In [ ]:
# @tool
# def router_tool(question):
#   """Router Function"""
#   if 'self-attention' in question:
#     return 'vectorstore'
#   else:
#     return 'web_search'

@tool
def router_tool(question: str) -> str:
  """
  Router Function
  Определяет, следует ли направить вопрос пользователя в vectorstore или в web_search.

  Args:
    question (str): Вопрос пользователя для маршрутизации.

  Returns:
    str: 'vectorstore', если вопрос касается 'self-attention', иначе 'web_search'.
  """
  # Приводим к нижнему регистру для надежности сравнения
  if 'self-attention' in question.lower():
    return 'vectorstore'
  else:
    return 'web_search' # Убедитесь, что это соответствует ожидаемому 'web_search' в задаче

## Create Agents to work with

In [ ]:
Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web_search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web_search."
    "Use the vectorstore for questions on concepts related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web_search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
  tools=[router_tool]
)

In [ ]:
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [ ]:
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [ ]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [ ]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer generated is not relevant then perform a web search using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

## Defining Tasks

In [ ]:
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'web_search' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'web_search' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [ ]:
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'web_search'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'web_search' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a clear and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)

In [ ]:
grader_task = Task(
    description=("Based on the response from the retriever task for the question {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [ ]:
hallucination_task = Task(
    description=("Based on the response from the grader task for the question {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

answer_task = Task(
    description=("Based on the response from the hallucination task for the question {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'web_search' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

## Define a flow for the use case

In [ ]:
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)

## Ask any query

In [ ]:
#inputs ={"question":"How does self-attention mechanism help large language models?"}
inputs ={"question":"Что такое CrewAI и как им пользоваться?"}

## Kick off the agent pipeline

In [ ]:
result = rag_crew.kickoff(inputs=inputs)

## Obtain the final response

In [ ]:
print(result)